In [ ]:
import boto3

client = boto3.client('elbv2')

In [ ]:
response = client.create_target_group(
    Name='geoff',
    HealthCheckEnabled=True,
    HealthCheckPath='/health',
    TargetType='lambda'
)

In [ ]:
print(response)